In [2]:
import numpy as np

def load_poses(file_path):
    """
    Load the poses from SemanticKITTI 'poses.txt' file.
    Args:
        file_path (str): Path to the poses.txt file.
    Returns:
        poses (list of numpy.ndarray): List of 4x4 transformation matrices.
    """
    poses = []
    with open(file_path, 'r') as f:
        for line in f:
            values = list(map(float, line.strip().split()))
            pose = np.eye(4)  # 4x4 identity matrix
            pose[:3, :4] = np.array(values).reshape(3, 4)  # Insert 3x4 pose
            poses.append(pose)
    return poses

# Example: Load poses.txt
file_path = "./../daeyoung/SemanticKITTI/sequences/07/poses.txt"  # 파일 경로를 입력하세요
poses = load_poses(file_path)

# 첫 번째 프레임의 4x4 변환 행렬 출력
print("First frame pose matrix:")
print(poses[0])

First frame pose matrix:
[[ 1.00000e+00  0.00000e+00  1.67347e-10  0.00000e+00]
 [ 9.31323e-10  1.00000e+00  0.00000e+00  0.00000e+00]
 [-4.14730e-10  0.00000e+00  1.00000e+00  0.00000e+00]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]


In [3]:
print(poses[1])

[[ 9.99984e-01 -5.42030e-06 -5.65610e-03 -6.19824e-03]
 [ 9.20333e-06  1.00000e+00  6.68768e-04 -1.29639e-02]
 [ 5.65609e-03 -6.68809e-04  9.99984e-01  7.54008e-02]
 [ 0.00000e+00  0.00000e+00  0.00000e+00  1.00000e+00]]
